In [1]:
import pickle

with open("preprocessed/text_object", 'rb') as f:
    text_obj = pickle.load(f)
    f.close()

AttributeError: Can't get attribute 'CorpusToEmbedding' on <module '__main__'>

In [2]:
from utils.text_preprocessor import CorpusToEmbedding

import pandas as pd

df = pd.read_csv("./preprocessed/train.csv")

import pickle

with open("preprocessed/text_object", 'rb') as f:
    text_obj = pickle.load(f)
    f.close()

In [5]:
df["text_tokens"] = text_obj.tokens
df["labels_tokens"] = text_obj.labels

In [7]:
from torchvision import datasets, transforms
import torch

In [8]:
import numpy as np

df["new_labels"] = np.where(df.label == 0, 1, 0)
df["new_id"] = df.id.astype(str)

In [13]:
import ast

ast.literal_eval(df.labels[0])

['person', 'person']

In [17]:
df = df.sort_values(["new_labels", "new_id"])

In [21]:
texts = torch.utils.data.TensorDataset(torch.tensor(df.text_tokens), torch.tensor(df.new_labels))

In [14]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

images = datasets.ImageFolder("./preprocessed/img", transform=transformations)

In [23]:
from torch.utils.data.sampler import RandomSampler

random_sampler = RandomSampler(images)

In [28]:
image_loader = torch.utils.data.DataLoader(
    images, batch_size=32, sampler=random_sampler
)
text_loader = torch.utils.data.DataLoader(texts, batch_size=32, sampler=random_sampler)

In [84]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        drp = 0.1
        n_classes = 2

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)

        self.cnn_layer1 = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # # Layer 3
            # nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0),
            # # Out: 62, 29
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            # # Out: 30, 14
            # nn.Dropout(drp),
        )

        self.fc1 = nn.Linear(224 * 224 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)

    def forward(self, x):
        # x = torch.unsqueeze(x, dim=1)
        x = x[0]
        out1 = self.cnn_layer1(x)
        # out2 = self.cnn_layer2(x)
        # full_out = torch.cat([out1, out2], dim=1)
        # full_out = full_out.view(full_out.size(0), -1)
        x = torch.flatten(out1, start_dim=1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.dropout(self.relu(self.fc4(x)))
        x = self.dropout(self.relu(self.fc5(x)))
        out = self.out(x)
        return out

In [85]:
model = CNN()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

# loss_fn = nn.

for (i, j) in zip(image_loader, text_loader):
    image = i[0].to(device)
    text = j[0].to(device)
    output = model([image, text]) # hacky way to do this?
    print(output)
    break

tensor([[ 0.0377, -0.0567],
        [ 0.0213, -0.0545],
        [ 0.0201, -0.0577],
        [-0.0030, -0.0609],
        [ 0.0281, -0.0857],
        [ 0.0346, -0.0577],
        [ 0.0344, -0.0543],
        [ 0.0251, -0.0670],
        [ 0.0174, -0.0674],
        [ 0.0093, -0.0913],
        [ 0.0175, -0.0664],
        [ 0.0301, -0.0611],
        [ 0.0114, -0.0724],
        [ 0.0168, -0.0580],
        [ 0.0249, -0.0494],
        [ 0.0398, -0.0498],
        [ 0.0354, -0.0446],
        [ 0.0273, -0.0801],
        [ 0.0183, -0.0533],
        [ 0.0201, -0.0730],
        [ 0.0382, -0.0526],
        [ 0.0202, -0.0549],
        [ 0.0263, -0.0594],
        [ 0.0206, -0.0547],
        [ 0.0315, -0.0579],
        [ 0.0322, -0.0582],
        [ 0.0209, -0.0683],
        [ 0.0281, -0.0615],
        [ 0.0273, -0.0694],
        [ 0.0265, -0.0597],
        [ 0.0229, -0.0601],
        [ 0.0236, -0.0645]], device='cuda:0', grad_fn=<AddmmBackward0>)
